In [ ]:

# !pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
# !pip install transformers

In [ ]:
import torch

print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    try:
        print(f"CUDA Device Count: {torch.cuda.device_count()}")
        print(f"CUDA Device Name: {torch.cuda.get_device_name(0)}")
    except Exception as e:
        print(f"Note: CUDA is available but needs kernel restart to initialize properly")
else:
    print("CUDA not available - running on CPU")

In [ ]:
import tkinter as tk
from tkinter import scrolledtext, messagebox
import threading

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

from transformers import pipeline

# Load a conversational text generation model (better for chatbot)
chatbot_pipeline = pipeline(
    "text-generation",
    model="microsoft/DialoGPT-medium",
    device=0 if torch.cuda.is_available() else -1
)

# System prompt to guide the chatbot's behavior
system_prompt = """You are a helpful, knowledgeable teacher who assists students with their questions. You explain concepts clearly, use examples, and encourage learning."""

class ChatbotGUI:
    def __init__(self, root):
        self.root = root
        self.root.title("AI Teacher Chatbot")
        self.root.geometry("700x600")
        self.root.configure(bg="#f0f0f0")
        self.conversation_history = []
        
        # Title Label
        title_label = tk.Label(root, text="AI Teacher Chatbot", font=("Arial", 18, "bold"), 
                               bg="#f0f0f0", fg="#333")
        title_label.pack(pady=10)
        
        # Chat display area
        self.chat_display = scrolledtext.ScrolledText(root, wrap=tk.WORD, width=80, height=25,
                                                       font=("Arial", 10), bg="#ffffff", 
                                                       fg="#000000", state=tk.DISABLED)
        self.chat_display.pack(padx=10, pady=10)
        
        # Configure tags for styling
        self.chat_display.tag_config("user", foreground="#0066cc", font=("Arial", 10, "bold"))
        self.chat_display.tag_config("bot", foreground="#cc0000", font=("Arial", 10, "bold"))
        
        # Input frame
        input_frame = tk.Frame(root, bg="#f0f0f0")
        input_frame.pack(padx=10, pady=5, fill=tk.X)
        
        # Question input
        self.question_entry = tk.Entry(input_frame, font=("Arial", 11), width=50)
        self.question_entry.pack(side=tk.LEFT, padx=(0, 10), ipady=5)
        self.question_entry.bind("<Return>", lambda e: self.send_question())
        
        self.send_button = tk.Button(input_frame, text="Ask Question", font=("Arial", 11, "bold"),
                                     bg="#4CAF50", fg="white", command=self.send_question,
                                     cursor="hand2", padx=15, pady=5)
        self.send_button.pack(side=tk.LEFT)
        
      
        clear_button = tk.Button(input_frame, text="Clear Chat", font=("Arial", 11),
                                bg="#ff6b6b", fg="white", command=self.clear_chat,
                                cursor="hand2", padx=15, pady=5)
        clear_button.pack(side=tk.LEFT, padx=10)
        
        self.display_message("Bot", "Hello! I'm your AI teacher. Ask me anything and I'll do my best to help you learn!")
        
    def display_message(self, sender, message):
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.insert(tk.END, f"{sender}: ", sender.lower())
        self.chat_display.insert(tk.END, f"{message}\n\n")
        self.chat_display.see(tk.END)
        self.chat_display.config(state=tk.DISABLED)
    
    def send_question(self):
        question = self.question_entry.get().strip()
        if not question:
            messagebox.showwarning("Empty Question", "Please enter a question!")
            return
     
        
        self.display_message("You", question)
        self.question_entry.delete(0, tk.END)
        
        
        self.send_button.config(state=tk.DISABLED, text="Processing...")
        
        
        thread = threading.Thread(target=self.get_answer, args=(question,))
        thread.start()
    
    def get_answer(self, question):
        try:
            # Build conversation context
            self.conversation_history.append(question)
            conversation = " ".join(self.conversation_history[-6:])  # Keep last 6 turns
            
            # Generate response
            result = chatbot_pipeline(
                conversation,
                max_length=len(conversation.split()) + 50,
                num_return_sequences=1,
                temperature=0.7,
                do_sample=True,
                pad_token_id=chatbot_pipeline.tokenizer.eos_token_id
            )
            
            # Extract the new response (everything after the input)
            full_response = result[0]['generated_text']
            response = full_response[len(conversation):].strip()
            
            # If response is empty or too short, provide a fallback
            if len(response) < 5:
                response = self.get_fallback_response(question)
            
            self.conversation_history.append(response)
                
        except Exception as e:
            response = f"Sorry, I encountered an error: {str(e)}"
        
        
        self.root.after(0, self.display_bot_response, response)
    
    def get_fallback_response(self, question):
        """Provide helpful fallback responses"""
        question_lower = question.lower()
        
        if any(word in question_lower for word in ["physics", "science"]):
            return "Physics is fascinating! It helps us understand how the universe works, from tiny atoms to massive galaxies. What specific area of physics interests you?"
        elif any(word in question_lower for word in ["math", "mathematics"]):
            return "Mathematics is the language of patterns and logic! It's fundamental to science, engineering, and everyday problem-solving. What math topic would you like to explore?"
        elif any(word in question_lower for word in ["hi", "hello", "hey"]):
            return "Hello! I'm here to help you learn. Feel free to ask me about any subject - math, science, history, or anything else you're curious about!"
        else:
            return "That's an interesting question! I'm here to help you learn. Could you provide more details or ask about a specific topic?"
    
    def display_bot_response(self, response):
        self.display_message("Bot", response)
        self.send_button.config(state=tk.NORMAL, text="Ask Question")
    
    def clear_chat(self):
        self.chat_display.config(state=tk.NORMAL)
        self.chat_display.delete(1.0, tk.END)
        self.chat_display.config(state=tk.DISABLED)
        self.conversation_history = []
        self.display_message("Bot", "Chat cleared! Ask me a new question.")


root = tk.Tk()
app = ChatbotGUI(root)
root.mainloop()
